In [1]:
import numpy as np
import pandas as pd
# fix some weird version hell
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as pdr
import pandas_profiling as pdf
import matplotlib.pyplot as plt
%matplotlib inline


# Data prep and scrubbing

### Load data

In [2]:
# load CSV and do initial cleaning 
# commented out because after loading we save to pickle

#print ("Load data from CSV")
#%time df = pd.read_csv('data.csv')
#print(df.shape)
# 'B', 'C' values in RET column
#df = df.loc[df["RET"] != "B"]
#df = df.loc[df["RET"] != "C"]
# print(df.shape)
#df["RET"] = df["RET"].astype(float)
# # takes few minutes
#print ("Convert date strings to datetime")
#%time df['DATETIME']=pd.to_datetime(df['realdate'])
#df.to_pickle('data_initial.pickle')

In [3]:
df = pd.read_pickle('data_initial.pickle')
print(df.shape)

(2781287, 12)


In [4]:
df.tail()

,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank,DATETIME
2783235,11372,79066,1.337235,30.0,9.0,0.105532,31DEC2017,2.872170e+06,-0.414414,0,0.0,2017-12-31
2783236,271987,13279,0.138449,48.0,1.0,-0.151961,31DEC2017,2.554080e+06,-0.943720,0,0.0,2017-12-31
2783237,13460,14135,-0.553076,30.0,0.0,0.078921,31DEC2017,1.804202e+06,-0.733194,0,0.0,2017-12-31
2783238,275799,15379,1.255997,48.0,9.0,-0.175000,31DEC2017,1.620800e+06,-0.711223,0,0.0,2017-12-31
2783239,186953,13844,0.813435,28.0,7.0,-0.082857,31DEC2017,1.045240e+06,-0.858621,0,0.0,2017-12-31


In [5]:
# before doing any scrubbing, compute 3-month returns

# Clean up these columns that should always be integers
for colname in ["gvkey",
                "PERMNO"]:
    df[colname] = df[colname].astype(int)

# group by gvkey and shirt
dfsort = df.sort_values(["gvkey","DATETIME"])
dfsort.head()
dfsort["RET1"] = dfsort.groupby("gvkey").shift(-1)["RET"]
dfsort["RET2"] = dfsort.groupby("gvkey").shift(-2)["RET"]
# compute 3-month returns
dfsort["RET3M"] = (1 + dfsort["RET"]) * (1 + dfsort["RET1"]) * (1 + dfsort["RET2"]) -1
%time dfsort.to_pickle('data_ret3.pickle')

CPU times: user 880 ms, sys: 348 ms, total: 1.23 s
Wall time: 829 ms


In [6]:
df = pd.read_pickle('data_ret3.pickle')
print(df.shape)
df[["DATETIME", "gvkey", "RET", "RET1", "RET2", "RET3M"]].tail()

(2781287, 15)


,DATETIME,gvkey,RET,RET1,RET2,RET3M
2765341,2017-08-31,317427,0.054035,-0.029960,0.036376,0.059649
2769141,2017-09-30,317427,-0.029960,0.036376,0.088079,0.093874
2773005,2017-10-31,317427,0.036376,0.088079,0.029215,0.160604
2776781,2017-11-30,317427,0.088079,0.029215,NaN,NaN
2780547,2017-12-31,317427,0.029215,NaN,NaN,NaN


In [7]:
print("Truncate before 1963")
print(df.shape)
df = df.loc[df["DATETIME"] > pd.to_datetime('1963-Jan-01')]
print(df.shape)
%time df.to_pickle('data_trunc.pickle')


Truncate before 1963
(2781287, 15)
(2717467, 15)
CPU times: user 192 ms, sys: 332 ms, total: 524 ms
Wall time: 834 ms


In [8]:
print("Read pickle")
df = pd.read_pickle('data_trunc.pickle')
print(df.shape)
df.tail()

Read pickle
(2717467, 15)


,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank,DATETIME,RET1,RET2,RET3M
2765341,317427,15920,0.937019,32.0,8.0,0.054035,31AUG2017,2.542656e+09,0.594350,8,7.0,2017-08-31,-0.029960,0.036376,0.059649
2769141,317427,15920,0.937019,32.0,8.0,-0.029960,30SEP2017,2.588772e+09,0.218991,6,7.0,2017-09-30,0.036376,0.088079,0.093874
2773005,317427,15920,0.937019,32.0,8.0,0.036376,31OCT2017,2.511212e+09,0.270728,7,6.0,2017-10-31,0.088079,0.029215,0.160604
2776781,317427,15920,0.937019,32.0,8.0,0.088079,30NOV2017,2.602561e+09,0.185517,4,7.0,2017-11-30,0.029215,NaN,NaN
2780547,317427,15920,0.937019,32.0,8.0,0.029215,31DEC2017,2.831793e+09,0.604676,8,7.0,2017-12-31,NaN,NaN,NaN


In [9]:
# a number of rows with NaN bm_orig, bm_rank
dfnas = df.loc[df["bm_orig"].isnull()].sort_values(['PERMNO', 'gvkey', 'DATETIME'])
dfnas.head()

,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank,DATETIME,RET1,RET2,RET3M
924413,19049,10002,NaN,44.0,NaN,0.037037,31JUL1987,15862500.0,0.064039,5,2.0,1987-07-31,-0.035714,-0.074074,-0.074074
929428,19049,10002,NaN,44.0,NaN,-0.035714,31AUG1987,16450000.0,0.014085,4,2.0,1987-08-31,-0.074074,-0.200000,-0.285714
934508,19049,10002,NaN,44.0,NaN,-0.074074,30SEP1987,15862500.0,0.051643,4,2.0,1987-09-30,-0.200000,0.012500,-0.250000
939680,19049,10002,NaN,44.0,NaN,-0.200000,31OCT1987,14687500.0,-0.052632,2,2.0,1987-10-31,0.012500,-0.012346,-0.200000
944645,19049,10002,NaN,44.0,NaN,0.012500,30NOV1987,11750000.0,-0.065421,2,2.0,1987-11-30,-0.012346,0.037500,0.037500


In [10]:
# filter rows with no bm_orig
df = df.loc[df["bm_orig"].notnull()]
df.count()

gvkey         2666442
PERMNO        2666442
bm_orig       2666442
ind_code      2654832
bm_rank       2666442
RET           2664708
realdate      2666442
lag_me        2665113
cum_return    2666442
mom_rank      2666442
me_rank       2665113
DATETIME      2666442
RET1          2643585
RET2          2622570
RET3M         2620141
dtype: int64

In [11]:
# some rows with no RET3M or ME
dfnas = df.loc[(df["RET3M"].isnull()) | (df["lag_me"].isnull())]
print(dfnas.shape)
dfnas.head()

(47112, 15)


,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank,DATETIME,RET1,RET2,RET3M
461952,1000,25881,1.081380,15.0,4.0,0.013699,31MAY1978,20312250.0,0.563992,8,3.0,1978-05-31,0.081081,NaN,NaN
465684,1000,25881,1.081380,15.0,4.0,0.081081,30JUN1978,20590500.0,0.509596,7,3.0,1978-06-30,NaN,NaN,NaN
860823,1001,10015,0.359772,43.0,2.0,0.008403,30JUN1986,59276875.0,1.212766,9,5.0,1986-06-30,-0.057667,NaN,NaN
865550,1001,10015,0.228607,43.0,2.0,-0.057667,31JUL1986,59775000.0,1.288462,9,5.0,1986-07-31,NaN,NaN,NaN
1051680,1003,10031,-0.064273,41.0,0.0,0.000000,31JUL1989,3018375.0,-0.718750,0,0.0,1989-07-31,0.000000,NaN,NaN


In [12]:
# filter rows with no RET3M or lag_me
df = df.loc[df["RET3M"].notnull()]
df = df.loc[df["lag_me"].notnull()]
df.count()

gvkey         2619330
PERMNO        2619330
bm_orig       2619330
ind_code      2608122
bm_rank       2619330
RET           2619330
realdate      2619330
lag_me        2619330
cum_return    2619330
mom_rank      2619330
me_rank       2619330
DATETIME      2619330
RET1          2619330
RET2          2619330
RET3M         2619330
dtype: int64

In [13]:
# set ind code to -1 where missing (unknown industry)
df.loc[df["ind_code"].isnull(), ["ind_code"]] = -1
df.count()

gvkey         2619330
PERMNO        2619330
bm_orig       2619330
ind_code      2619330
bm_rank       2619330
RET           2619330
realdate      2619330
lag_me        2619330
cum_return    2619330
mom_rank      2619330
me_rank       2619330
DATETIME      2619330
RET1          2619330
RET2          2619330
RET3M         2619330
dtype: int64

In [14]:
# double-check dates for each gvkey
uniquegvkeys = sorted(list(df.gvkey.unique()))
uniquegvkeys
print("Unique gvkeys: %d" % len(uniquegvkeys))
uniquedates = sorted(list(df.DATETIME.unique()))
uniquedates = [str(d)[:10] for d in uniquedates]
date_dict = dict(zip(uniquedates, range(len(uniquedates))))
date_dict

# so for each gvkey
# get first date
# get last date
# count number of dates
# count number of dates in uniquedates such that date >=firstdate and date <=lastdate
# report if they don't match

gb = df.groupby(["gvkey"])
gvkey_min = gb.min()["DATETIME"]
gvkey_max = gb.max()["DATETIME"]
gvkey_count = gb.count()["DATETIME"]

gvkey_df = pd.DataFrame(dict(firstdate = gvkey_min, lastdate = gvkey_max, count = gvkey_count))

def count_dates(row):
    firstdate = str(row['firstdate'])[:10]
    lastdate = str(row['lastdate'])[:10]
    f = date_dict[firstdate]
    l = date_dict[lastdate]
    return(l-f+1)

gvkey_df['monthcount'] = gvkey_df.apply(count_dates, axis=1)

Unique gvkeys: 20959


In [15]:
missing_df = gvkey_df[gvkey_df["count"] < gvkey_df["monthcount"]].reset_index()
missing_df.tail()
# these are ones where there was a corporate action? or some missing data?
# only 5, but messes with their 3m returns


,gvkey,firstdate,lastdate,count,monthcount
2207,178906,2009-01-31,2017-10-31,65,106
2208,179817,2009-06-30,2017-10-31,79,101
2209,180272,2009-08-31,2017-08-31,69,97
2210,184545,2012-01-31,2017-10-31,52,70
2211,185193,2012-04-30,2017-10-31,48,67


In [16]:
### Load CRSP company names, to make more sense of data and sanity check
#https://wrds-web.wharton.upenn.edu/wrds/ds/crsp/index.cfm
# read header file with PERMNO-SYMBOL mapping
# fixed width file
df_header = pd.read_fwf('cheadfile.dat')
df_header.head()
symbol_lookup = dict(zip(df_header.PERMNO, df_header.SYMBOL))


In [17]:
def get_symbol(row):
    return symbol_lookup.get(row["gvkey"])
                             
missing_df["SYMBOL"] = missing_df.apply(get_symbol, axis=1)
missing_df.sort_values("lastdate", ascending=False).head()

,gvkey,firstdate,lastdate,count,monthcount,SYMBOL
0,1004,1973-05-31,2017-10-31,519,534,None
1190,10002,1971-01-31,2017-10-31,491,562,BTFG
1210,10187,1974-01-31,2017-10-31,514,526,HPLX
1212,10195,1972-06-30,2017-10-31,530,545,CTCOB
1215,10235,1984-10-31,2017-10-31,321,397,FOTO


In [18]:
# strange things are afoot
(df[df.gvkey == 10235]).tail(23)
# looking at original file to make sure, seems imported correctly 
# grep ^010235, data.csv
# gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank
# 010235,65429,1.0335767755,8,7,-0.109464,31OCT2011,126708966.61,0.1627296415,6,3
# 010235,65429,1.0335767755,8,7,B,30NOV2011,112838879.17,0.2046703698,8,2
# 010235,65429,1.0335767755,8,7,B,31DEC2011,,0.107801387,6,
# 010235,65429,0.1274307784,8,1,-0.129450,31JAN2017,236329391.67,0.6313746285,9,3
# 010235,65429,0.1274307784,8,1,0.060719,28FEB2017,205736572.22,0.5475794259,7,2


,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,me_rank,DATETIME,RET1,RET2,RET3M
2446291,10235,65429,1.901253,8.0,9.0,-0.044619,31OCT2010,1.100938e+08,0.025958,5,2.0,2010-10-31,-0.031593,0.025532,-0.051180
2450471,10235,65429,1.901253,8.0,9.0,-0.031593,30NOV2010,1.051814e+08,0.083926,4,2.0,2010-11-30,0.025532,-0.125864,-0.131867
2454613,10235,65429,1.901253,8.0,9.0,0.025532,31DEC2010,1.018584e+08,0.011111,3,2.0,2010-12-31,-0.125864,0.053797,-0.055319
2458758,10235,65429,1.901253,8.0,9.0,-0.125864,31JAN2011,1.044590e+08,-0.024896,3,2.0,2011-01-31,0.053797,0.001502,-0.077455
2462905,10235,65429,1.901253,8.0,9.0,0.053797,28FEB2011,9.131136e+07,-0.075448,1,2.0,2011-02-28,0.001502,0.191904,0.257911
2466980,10235,65429,1.901253,8.0,9.0,0.001502,31MAR2011,9.622368e+07,-0.364824,0,2.0,2011-03-31,0.191904,0.184906,0.414415
2471047,10235,65429,1.901253,8.0,9.0,0.191904,30APR2011,9.636816e+07,-0.331325,0,2.0,2011-04-30,0.184906,0.144374,0.616193
2474999,10235,65429,1.901253,8.0,9.0,0.184906,31MAY2011,1.148616e+08,-0.267838,1,2.0,2011-05-31,0.144374,-0.031540,0.313208
2478935,10235,65429,1.901253,8.0,9.0,0.144374,30JUN2011,1.361002e+08,0.110335,3,2.0,2011-06-30,-0.031540,-0.151341,-0.059448
2482864,10235,65429,1.033577,8.0,7.0,-0.031540,31JUL2011,1.557494e+08,0.326761,5,3.0,2011-07-31,-0.151341,-0.010158,-0.186456


In [19]:
# Merge symbol into data
# left outer join
print(df.shape)
df_merged = pd.merge(df, df_header, how='left', on='PERMNO')
print(df_merged.shape)
df_merged.head()

(2619330, 15)
(2619330, 21)


,gvkey,PERMNO,bm_orig,ind_code,bm_rank,RET,realdate,lag_me,cum_return,mom_rank,...,DATETIME,RET1,RET2,RET3M,PERMC,CUSIP,COMPANY NAME,SYMBOL,X,START-END
0,1000,25881,0.481379,15.0,3.0,0.393939,31DEC1971,10951875.0,-0.458333,0,...,1971-12-31,0.000000,-0.086957,0.272726,23369.0,00003210,A & E PLASTIK PAK INC,AE,2.0,19701130-19780630
1,1000,25881,0.481379,15.0,3.0,0.000000,31JAN1972,15266250.0,-0.587500,0,...,1972-01-31,-0.086957,0.071429,-0.021739,23369.0,00003210,A & E PLASTIK PAK INC,AE,2.0,19701130-19780630
2,1000,25881,0.481379,15.0,3.0,-0.086957,29FEB1972,15266250.0,-0.361111,0,...,1972-02-29,0.071429,0.266667,0.239131,23369.0,00003210,A & E PLASTIK PAK INC,AE,2.0,19701130-19780630
3,1000,25881,0.481379,15.0,3.0,0.071429,31MAR1972,13938750.0,-0.483146,0,...,1972-03-31,0.266667,-0.070175,0.261906,23369.0,00003210,A & E PLASTIK PAK INC,AE,2.0,19701130-19780630
4,1000,25881,0.481379,15.0,3.0,0.266667,30APR1972,14934375.0,-0.461538,0,...,1972-04-30,-0.070175,-0.094340,0.066667,23369.0,00003210,A & E PLASTIK PAK INC,AE,2.0,19701130-19780630


In [20]:
# clean up column names
collist = list(df_merged.columns)
collist = [s.upper() for s in collist]
collist = [s.replace(' ', '_') for s in collist]
df_merged.columns = collist
df_merged.count()

                                                              


GVKEY           2619330
PERMNO          2619330
BM_ORIG         2619330
IND_CODE        2619330
BM_RANK         2619330
RET             2619330
REALDATE        2619330
LAG_ME          2619330
CUM_RETURN      2619330
MOM_RANK        2619330
ME_RANK         2619330
DATETIME        2619330
RET1            2619330
RET2            2619330
RET3M           2619330
PERMC           2619221
CUSIP           2619221
COMPANY_NAME    2619221
SYMBOL          2617361
X               2619221
START-END       2619221
dtype: int64

In [21]:
# delete these 5 stocks
df = df[df["gvkey"] != 1004]
df = df[df["gvkey"] != 10002]
df = df[df["gvkey"] != 10187]
df = df[df["gvkey"] != 10195]
df = df[df["gvkey"] != 10235]


In [22]:
df_merged.columns

Index(['GVKEY', 'PERMNO', 'BM_ORIG', 'IND_CODE', 'BM_RANK', 'RET', 'REALDATE',
       'LAG_ME', 'CUM_RETURN', 'MOM_RANK', 'ME_RANK', 'DATETIME', 'RET1',
       'RET2', 'RET3M', 'PERMC', 'CUSIP', 'COMPANY_NAME', 'SYMBOL', 'X',
       'START-END'],
      dtype='object')

In [23]:
# keep these columns
df_merged=df_merged[["DATETIME",
                     "GVKEY",
                     "SYMBOL",
                     "IND_CODE",
                     "BM_ORIG",
                     "BM_RANK",
                     "CUM_RETURN",
                     "MOM_RANK",
                     "RET",
                     "RET3M",
                     "LAG_ME"
                    ]]

df_merged.head()



,DATETIME,GVKEY,SYMBOL,IND_CODE,BM_ORIG,BM_RANK,CUM_RETURN,MOM_RANK,RET,RET3M,LAG_ME
0,1971-12-31,1000,AE,15.0,0.481379,3.0,-0.458333,0,0.393939,0.272726,10951875.0
1,1972-01-31,1000,AE,15.0,0.481379,3.0,-0.587500,0,0.000000,-0.021739,15266250.0
2,1972-02-29,1000,AE,15.0,0.481379,3.0,-0.361111,0,-0.086957,0.239131,15266250.0
3,1972-03-31,1000,AE,15.0,0.481379,3.0,-0.483146,0,0.071429,0.261906,13938750.0
4,1972-04-30,1000,AE,15.0,0.481379,3.0,-0.461538,0,0.266667,0.066667,14934375.0


In [24]:
# clean up ints in IND_CODE BM_RANK, MOM_RANK, ME_RANK
for colname in ["GVKEY",
                "IND_CODE",
                "BM_RANK",
                "MOM_RANK",
               ]:
    df_merged[colname] = df_merged[colname].astype(int)
#df.a = df.a.astype(float)
#df_merged.columns
df_merged.head()

,DATETIME,GVKEY,SYMBOL,IND_CODE,BM_ORIG,BM_RANK,CUM_RETURN,MOM_RANK,RET,RET3M,LAG_ME
0,1971-12-31,1000,AE,15,0.481379,3,-0.458333,0,0.393939,0.272726,10951875.0
1,1972-01-31,1000,AE,15,0.481379,3,-0.587500,0,0.000000,-0.021739,15266250.0
2,1972-02-29,1000,AE,15,0.481379,3,-0.361111,0,-0.086957,0.239131,15266250.0
3,1972-03-31,1000,AE,15,0.481379,3,-0.483146,0,0.071429,0.261906,13938750.0
4,1972-04-30,1000,AE,15,0.481379,3,-0.461538,0,0.266667,0.066667,14934375.0


In [25]:
# truncate to only stocks > median market cap
# could compute median from this file but let's use standard breakpoints from
# http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_me_breakpoints.html

breakpoints = pd.read_csv('ME_Breakpoints.CSV')

# convert 201801 to datetime
from calendar import monthrange
datelist = [dateno * 100 + monthrange(dateno // 100, dateno % 100)[1] for dateno in breakpoints.month]
breakpoints["date"] = [pd.to_datetime(str(d)) for d in datelist]

# merge breakpoints into df_merged
breakpoints["MEDIAN_ME"] = breakpoints[["percentile50"]] * 1000000
breakpoints = breakpoints[["date", "MEDIAN_ME"]]
df_merged = pd.merge(df_merged, breakpoints, how='left', left_on='DATETIME', right_on='date')
df_merged.shape

(2619330, 13)

In [26]:
# do the truncation
df_merged = df_merged.loc[df_merged["LAG_ME"] > df_merged["MEDIAN_ME"]]
df_merged.shape

(577153, 13)

In [27]:
pdf.ProfileReport(df_merged)
# Note distribution of BM_RANK
# interesting that BM_RANK has 8.5% of 0s, should be 10% in top decile
# we culled < 50% by market cap
# bottom half has more than its share of cheap stocks
# consistent with notion of small stock premium
# bottom half has more than its share of low momentum


Number of variables,14
Number of observations,577153
Total Missing (%),0.0%
Total size in memory,61.6 MiB
Average record size in memory,112.0 B
Numeric,11
Categorical,1
Boolean,0
Date,2
Text (Unique),0
Rejected,0


In [28]:
# since the low market cap stocks are low momentum, low p/b
# culling them unbalances deciles
# re-bucket deciles
# for each timestamp, compute new buckets

uniquedates = sorted(df_merged["DATETIME"].unique())
uniquedates[:10]

[numpy.datetime64('1963-01-31T00:00:00.000000000'),
 numpy.datetime64('1963-02-28T00:00:00.000000000'),
 numpy.datetime64('1963-03-31T00:00:00.000000000'),
 numpy.datetime64('1963-04-30T00:00:00.000000000'),
 numpy.datetime64('1963-05-31T00:00:00.000000000'),
 numpy.datetime64('1963-06-30T00:00:00.000000000'),
 numpy.datetime64('1963-07-31T00:00:00.000000000'),
 numpy.datetime64('1963-08-31T00:00:00.000000000'),
 numpy.datetime64('1963-09-30T00:00:00.000000000'),
 numpy.datetime64('1963-10-31T00:00:00.000000000')]

In [29]:
num_buckets = 10
# ideally do this before the cull, and compare the new buckets vs. actual
# should line up modulo rounding of bucket sizes

def bucket_df_by_col(df, col):
    # compute decile buckets for df.col
    # assign new column col_RANK containing decile bucket for each row
    newcol = col + "_RANK"
    df[newcol] = pd.qcut(df[col], num_buckets, labels=False)
    #df = df.assign(c=col.values)
    return df

dflist = []
for d in uniquedates:
    dftemp = df_merged[df_merged["DATETIME"]==d].copy()    
    dflist.append(bucket_df_by_col(dftemp, "BM_ORIG"))
dfnew=pd.concat(dflist)

dflist = []
for d in uniquedates:
    dftemp = dfnew[dfnew["DATETIME"]==d].copy()
    dflist.append(bucket_df_by_col(dftemp, "CUM_RETURN"))
dfnew= pd.concat(dflist)
dfnew.head()

,DATETIME,GVKEY,SYMBOL,IND_CODE,BM_ORIG,BM_RANK,CUM_RETURN,MOM_RANK,RET,RET3M,LAG_ME,date,MEDIAN_ME,BM_ORIG_RANK,CUM_RETURN_RANK
1016,1963-01-31,1010,ACF,25,1.443624,8,0.126256,9,0.047002,0.077724,112062625.0,1963-01-31,68440000.0,9,9
4326,1963-01-31,1040,AMF,6,0.448922,2,-0.477048,0,0.170732,0.041148,333719500.0,1963-01-31,68440000.0,2,0
4620,1963-01-31,1043,AM,41,0.255952,0,-0.390406,0,0.049505,-0.056931,332088000.0,1963-01-31,68440000.0,0,0
5016,1963-01-31,1045,AAL,40,1.383475,8,-0.130926,5,0.102041,0.082269,151593750.0,1963-01-31,68440000.0,9,4
8419,1963-01-31,1075,PNW,31,0.476919,2,-0.187948,3,0.060082,-0.002032,258187500.0,1963-01-31,68440000.0,3,2


In [30]:
pdf.ProfileReport(dfnew[["BM_ORIG_RANK", "CUM_RETURN_RANK"]])


Number of variables,3
Number of observations,577153
Total Missing (%),0.0%
Total size in memory,13.2 MiB
Average record size in memory,24.0 B
Numeric,3
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [31]:
# rename as VALUE_RANK, MOMENTUM_RANK, clean up column names and pickle
dfnew = dfnew[["DATETIME","GVKEY","SYMBOL","IND_CODE","BM_ORIG","BM_ORIG_RANK","CUM_RETURN","CUM_RETURN_RANK","RET","RET3M"]]
dfnew.columns=["DATETIME","GVKEY","SYMBOL","IND_CODE","VALUE","VALUE_RANK","MOMENTUM","MOMENTUM_RANK","RET","RET3"]
dfnew.head()

,DATETIME,GVKEY,SYMBOL,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3
1016,1963-01-31,1010,ACF,25,1.443624,9,0.126256,9,0.047002,0.077724
4326,1963-01-31,1040,AMF,6,0.448922,2,-0.477048,0,0.170732,0.041148
4620,1963-01-31,1043,AM,41,0.255952,0,-0.390406,0,0.049505,-0.056931
5016,1963-01-31,1045,AAL,40,1.383475,9,-0.130926,4,0.102041,0.082269
8419,1963-01-31,1075,PNW,31,0.476919,3,-0.187948,2,0.060082,-0.002032


In [32]:
# look up a stock and sanity check it vs. Yahoo
IBM = dfnew.loc[dfnew['SYMBOL']=='IBM']
IBM.head(20)


,DATETIME,GVKEY,SYMBOL,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3
688475,1963-01-31,6066,IBM,34,0.126757,0,-0.306857,0,0.086538,0.082187
688476,1963-02-28,6066,IBM,34,0.126757,0,-0.275333,1,-0.054277,0.097133
688477,1963-03-31,6066,IBM,34,0.126757,0,-0.206399,1,0.053158,0.196373
688478,1963-04-30,6066,IBM,34,0.126757,0,-0.243486,0,0.101544,0.038395
688479,1963-05-31,6066,IBM,34,0.126757,0,-0.065072,2,0.031267,-0.061109
688480,1963-06-30,6066,IBM,34,0.126757,0,0.188960,5,-0.085909,-0.066003
688481,1963-07-31,6066,IBM,34,0.114340,0,0.418594,7,-0.004011,0.038433
688482,1963-08-31,6066,IBM,34,0.114340,0,0.136728,3,0.025892,0.139495
688483,1963-09-30,6066,IBM,34,0.114340,0,0.103650,3,0.016301,0.093872
688484,1963-10-31,6066,IBM,34,0.114340,0,0.269149,4,0.092920,0.124570


In [33]:
IBM.head(20)


,DATETIME,GVKEY,SYMBOL,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3
688475,1963-01-31,6066,IBM,34,0.126757,0,-0.306857,0,0.086538,0.082187
688476,1963-02-28,6066,IBM,34,0.126757,0,-0.275333,1,-0.054277,0.097133
688477,1963-03-31,6066,IBM,34,0.126757,0,-0.206399,1,0.053158,0.196373
688478,1963-04-30,6066,IBM,34,0.126757,0,-0.243486,0,0.101544,0.038395
688479,1963-05-31,6066,IBM,34,0.126757,0,-0.065072,2,0.031267,-0.061109
688480,1963-06-30,6066,IBM,34,0.126757,0,0.188960,5,-0.085909,-0.066003
688481,1963-07-31,6066,IBM,34,0.114340,0,0.418594,7,-0.004011,0.038433
688482,1963-08-31,6066,IBM,34,0.114340,0,0.136728,3,0.025892,0.139495
688483,1963-09-30,6066,IBM,34,0.114340,0,0.103650,3,0.016301,0.093872
688484,1963-10-31,6066,IBM,34,0.114340,0,0.269149,4,0.092920,0.124570


In [34]:
IBM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658 entries, 688475 to 689132
Data columns (total 10 columns):
DATETIME         658 non-null datetime64[ns]
GVKEY            658 non-null int64
SYMBOL           658 non-null object
IND_CODE         658 non-null int64
VALUE            658 non-null float64
VALUE_RANK       658 non-null int64
MOMENTUM         658 non-null float64
MOMENTUM_RANK    658 non-null int64
RET              658 non-null float64
RET3             658 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 56.5+ KB


In [35]:
# duplicate 11-month return calc as a sanity check
IBM["cum2"] = IBM["RET"].rolling(11).apply(lambda x: np.prod(1+x)-1)
IBM[["DATETIME", "RET", "MOMENTUM", "cum2"]].to_csv("IBMcum.csv")

/home/ubuntu/anaconda2/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  
/home/ubuntu/anaconda2/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
print ("Pickle clean data")
%time dfnew.to_pickle('data_clean.pickle')

Pickle clean data
CPU times: user 48 ms, sys: 44 ms, total: 92 ms
Wall time: 87.3 ms


In [37]:
dfnew.describe()

,GVKEY,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3
count,577153.000000,577153.000000,577153.000000,577153.000000,577153.000000,577153.000000,577153.000000,577153.000000
mean,20958.232838,29.731468,0.608070,4.499110,0.207515,4.499110,0.008525,0.029794
std,38069.493665,12.686428,0.527173,2.874649,0.860592,2.874649,0.103594,0.181021
min,1004.000000,-1.000000,-58.420559,0.000000,-0.989569,0.000000,-0.981295,-0.997347
25%,4684.000000,21.000000,0.275950,2.000000,-0.046919,2.000000,-0.044025,-0.065051
50%,8240.000000,31.000000,0.492337,4.000000,0.132300,4.000000,0.007576,0.027340
75%,13003.000000,41.000000,0.826869,7.000000,0.344394,7.000000,0.060133,0.120978
max,316056.000000,48.000000,32.906680,9.000000,436.684199,9.000000,3.001658,5.131941


In [38]:
# compare some data to Yahoo 
sp500tr = pd.read_csv('SP500TR.csv')
sp500tr['Date'] = pd.to_datetime(sp500tr['Date'])
sp500tr = sp500tr.set_index("Date")
sp500tr = pd.DataFrame(sp500tr.groupby(pd.Grouper(freq="M")).last()['Adj Close'])
sp500tr.head()


,Adj Close
Date,
2014-05-31,3480.290039
2014-06-30,3552.179932
2014-07-31,3503.189941
2014-08-31,3643.330078
2014-09-30,3592.250000


In [39]:
IBMtr = pd.read_csv('IBMhist.csv')
IBMtr['Date'] = pd.to_datetime(IBMtr['Date'])
IBMtr = IBMtr.set_index("Date")
IBMtr = pd.DataFrame(IBMtr.groupby(pd.Grouper(freq="M")).last()['Adj Close'])
IBMtr.head()


,Adj Close
Date,
2014-05-31,155.676666
2014-06-30,153.067429
2014-07-31,161.849335
2014-08-31,163.341660
2014-09-30,161.243607


In [40]:
yahoodata = pd.merge(IBMtr, sp500tr, how='left', on='Date')
yahoodata.columns=['IBM_yahoo_adj_close', 'SP500TR_yahoo']
yahoodata.head()

,IBM_yahoo_adj_close,SP500TR_yahoo
Date,,
2014-05-31,155.676666,3480.290039
2014-06-30,153.067429,3552.179932
2014-07-31,161.849335,3503.189941
2014-08-31,163.341660,3643.330078
2014-09-30,161.243607,3592.250000


In [41]:
IBM = pd.merge(IBM, yahoodata, how='inner', left_on='DATETIME', right_on='Date')
IBM.head()

,DATETIME,GVKEY,SYMBOL,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3,cum2,IBM_yahoo_adj_close,SP500TR_yahoo
0,2014-05-31,6066,IBM,34,0.091118,0,-0.030280,1,-0.056039,-0.018611,-0.014318,155.676666,3480.290039
1,2014-06-30,6066,IBM,34,0.091118,0,-0.040685,0,-0.016761,0.049034,-0.050367,153.067429,3552.179932
2,2014-07-31,6066,IBM,34,0.133711,1,-0.014317,0,0.057373,0.053212,0.068894,161.849335,3503.189941
3,2014-08-31,6066,IBM,34,0.133711,1,-0.050366,0,0.009026,-0.137370,0.061594,163.341660,3643.330078
4,2014-09-30,6066,IBM,34,0.133711,1,0.068895,2,-0.012845,-0.150962,0.082868,161.243607,3592.250000


In [42]:
IBM['pct_change'] = IBM["IBM_yahoo_adj_close"].pct_change()
IBM['shift2'] = IBM.IBM_yahoo_adj_close.shift(2)
IBM['shift13'] = IBM.IBM_yahoo_adj_close.shift(13)
IBM['momentum_raw'] = IBM['shift2']/IBM['shift13']-1

#IBM[['RET', 'CUM_RETURN', 'IBM_yahoo_adj_close', 'SP500TR_yahoo',
#       'pct_change', 'shift1', 'shift12',  'CUM_RETURN', 'momentum_raw', 'SP_pct_change',
#       'SP_shift1', 'SP_shift12', 'momentum_excess_return']].tail(12)

IBM[['DATETIME','IBM_yahoo_adj_close', 'MOMENTUM', 'momentum_raw']].tail(14)


,DATETIME,IBM_yahoo_adj_close,MOMENTUM,momentum_raw
28,2016-09-30,146.832565,0.117162,0.128881
29,2016-10-31,142.062927,0.137251,0.148981
30,2016-11-30,151.307953,0.176729,0.188866
31,2016-12-31,154.824356,0.133327,0.144953
32,2017-01-31,162.780579,0.222333,0.235413
33,2017-02-28,169.050247,0.379332,0.394092
34,2017-03-31,163.710434,0.367582,0.381816
35,2017-04-30,150.689941,0.228615,0.241552
36,2017-05-31,144.890442,0.234728,0.247729
37,2017-06-30,146.029572,0.069128,0.069540


In [43]:
dfnew.groupby("DATETIME").mean().tail() # to check downstream stuff

,GVKEY,IND_CODE,VALUE,VALUE_RANK,MOMENTUM,MOMENTUM_RANK,RET,RET3
DATETIME,,,,,,,,
2017-06-30,54862.023483,31.803327,0.483241,4.500000,0.226120,4.500000,0.014752,0.027915
2017-07-31,55125.140610,31.820059,0.422756,4.499508,0.247844,4.499508,0.017596,0.048007
2017-08-31,55072.109284,31.852031,0.422186,4.500000,0.206165,4.500000,-0.005292,0.047737
2017-09-30,54117.079637,31.868952,0.422276,4.500000,0.216643,4.500000,0.035185,0.089070
2017-10-31,53797.722664,31.833996,0.419970,4.498012,0.202488,4.498012,0.018005,0.061582
